In [1]:
import numpy as np
np.random.seed(123)
import pandas as pd
import subprocess
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from scipy.stats import skew, boxcox




/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

In [40]:
shift = 200
def transform(df):
    for i in range(df.shape[1]):
        df.ix[:,i] = np.log(df.ix[:,i]+shift)
    return df


In [6]:
train = pd.read_csv("./input/train.csv")
ids = pd.read_csv("./input/test.csv")['id']

In [60]:
train_x = pd.read_csv("./input/train_second_level_model.csv")
test_x = pd.read_csv("./input/test_second_level_model.csv")

train_y = np.log(train['loss'].values+shift)

In [61]:
train_x.shape[1]

12

In [62]:
for i in range(train_x.shape[1]):
    print mean_absolute_error(train_x.ix[:,i],train.loss)



1124.2522893
1125.11043344
1124.73870933
1125.09915962
1124.87963343
1124.45611729
1130.58667325
1130.40016748
1131.39946641
1132.36796706
1190.29358276
1186.37357485


In [63]:
xtrain = csr_matrix(transform(train_x))
xtest = csr_matrix(transform(test_x))

In [64]:
def mae(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true) - K.exp(y_pred)))

def nn_model(layer1=200,layer2=100,dropout1 = 0.4,dropout2=0.2, optimizer = 'adadelta'):
    model = Sequential()
    
    model.add(Dense(layer1, input_dim = xtrain.shape[1], init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(dropout1))
        
    model.add(Dense(layer2, init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(dropout2))
    
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'mae', optimizer = optimizer, metrics=[mae])
    return(model)

In [82]:
np.random.seed(2016)
nfolds = 5
folds = KFold(len(train_y), n_folds = nfolds, shuffle = True, random_state = 2016)

## train models
i = 0
nbags = 5
nepochs = 60
pred_oob = np.zeros(xtrain.shape[0])
pred_test = np.zeros(xtest.shape[0])

partial_evalutaion = open('temp_scores1.txt','a') 

for (inTr, inTe) in folds:
    xtr = xtrain[inTr]
    ytr = train_y[inTr]
    xte = xtrain[inTe]
    yte = train_y[inTe]
    pred = np.zeros(xte.shape[0])
    for j in range(nbags):
        print j
                
        callsback_list = [EarlyStopping(patience=10),\
                  ModelCheckpoint('keras-regressor-' + str(i+1) +'_'+ str(j+1) + '.check'\
                                  , monitor='val_loss', save_best_only=True, verbose=0)]
        
        model = nn_model(layer1=250,layer2=100,\
             dropout1 = 0.4,dropout2=0.2, \
             optimizer = 'adadelta')
        
        fit = model.fit_generator(generator = batch_generator(xtr, ytr, 128, True),
                                  nb_epoch = nepochs,
                                  samples_per_epoch = xtr.shape[0],
                                  verbose = 2,
                                  validation_data=(xte.todense(),yte),
                                  callbacks=callsback_list)
        
        fit = load_model('keras-regressor-' + str(i+1) + '_'+ str(j+1) + '.check')
        
        pred += np.exp(fit.predict_generator(generator = batch_generatorp(xte, 800, False), \
                                               val_samples = xte.shape[0])[:,0])-200
        
        pred_test += np.exp(fit.predict_generator(generator = batch_generatorp(xtest, 800, False), \
                                                    val_samples = xtest.shape[0])[:,0])-200
    pred /= nbags
    pred_oob[inTe] = pred
    score = mean_absolute_error(np.exp(yte)-200, pred)
    
    i += 1

    print('Fold ', i, '- MAE:', score)

    partial_evalutaion.write('Fold '+ str(i) + '- MAE:'+ str(score)+'\n')
    partial_evalutaion.flush()


partial_evalutaion.write('Total - MAE:'+ str(mean_absolute_error(np.exp(train_y)-200, pred_oob))+'\n')

print('Total - MAE:', mean_absolute_error(np.exp(train_y)-200, pred_oob))

0
Epoch 1/60
7s - loss: 1.8870 - mae: 8678.9379 - val_loss: 0.5019 - val_mae: 1747.6968
Epoch 2/60
8s - loss: 0.7320 - mae: 3721.6229 - val_loss: 0.4089 - val_mae: 1488.2590
Epoch 3/60
7s - loss: 0.6207 - mae: 2518.9037 - val_loss: 0.3986 - val_mae: 1322.2816
Epoch 4/60
7s - loss: 0.5647 - mae: 2129.3972 - val_loss: 0.3751 - val_mae: 1139.6498
Epoch 5/60
7s - loss: 0.5319 - mae: 1890.0873 - val_loss: 0.3687 - val_mae: 1130.2145
Epoch 6/60
7s - loss: 0.5126 - mae: 1744.6826 - val_loss: 0.3647 - val_mae: 1123.3800
Epoch 7/60
7s - loss: 0.4968 - mae: 1663.5865 - val_loss: 0.3855 - val_mae: 1197.6542
Epoch 8/60
7s - loss: 0.4848 - mae: 1594.8720 - val_loss: 0.3645 - val_mae: 1119.1134
Epoch 9/60
7s - loss: 0.4772 - mae: 1559.5711 - val_loss: 0.3672 - val_mae: 1126.1706
Epoch 10/60
7s - loss: 0.4711 - mae: 1517.8301 - val_loss: 0.3658 - val_mae: 1121.9227
Epoch 11/60
7s - loss: 0.4645 - mae: 1484.3993 - val_loss: 0.3724 - val_mae: 1152.0274
Epoch 12/60
7s - loss: 0.4603 - mae: 1459.8795 - v

In [83]:
total_score = mean_absolute_error(np.exp(train_y)-200, pred_oob)

In [84]:
total_score

1117.880379920515

In [85]:
df = pd.DataFrame({'loss': pred_oob})
df.to_csv(str(total_score) + 'preds_oob.csv', index = False)

pred_test /= (nfolds*nbags)
df = pd.DataFrame({'id': ids, 'loss': pred_test})
df.to_csv(str(total_score) + '_submission_keras_shift_perm.csv', index = False)